# Agentic retrieval using Azure AI Search and Azure AI Agent Service

Use this notebook to create an agentic retrieval pipeline built on Azure AI Search and an Azure AI Agent.

In this walkthrough, you will:

+ Create an "earth_at_night" search index
+ Load it with documents from a GitHub URL
+ Create a knowledge source that points to searchable content.
+ Create a knowledge agent on Azure AI Search that points to a knowledge source and an LLM for intelligent query planning
+ Create a Foundry agent in Azure AI Foundry to determine when queries are needed
+ Create a Azure AI Agent tool (client) to orchestrate all requests
+ Start a chat with the agent

This notebook is referenced in [Build an agentic retrieval pipeline in Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-pipeline).

This exercise differs from the [Agentic Retrieval Quickstart](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval) in how it uses Azure AI Agent to determine whether to retrieve data from the index, and how it uses an agent tool for orchestration.

## Prerequisites

+ Azure AI Search, basic tier or higher, in [any region that supports semantic ranker](https://learn.microsoft.com/azure/search/search-region-support#azure-public-regions).

+ Azure OpenAI, and you should have an **Azure AI Developer** role assignment to create a Foundry project.

+ An [Azure AI agent and Foundry project](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal), created in the Azure AI Foundry portal, with the basic setup, used for creating the Foundry agent.

+ A deployment of a [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) in your Foundry project. This notebook uses gpt-5-mini. We recommend 100,000 token capacity. You can find capacity and the rate limit in the model deployments list in the Azure AI Foundry portal.

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.13.7.

## Set up connections

Save the `sample.env` file as `.env` and then modify the environment variables to use your Azure endpoints. You need endpoints for:

+ Azure AI Search
+ Azure OpenAI
+ Azure AI Foundry project

You can find endpoints for Azure AI Search and Azure OpenAI in the [Azure portal](https://portal.azure.com).

You can find the project endpoint in the Azure AI Foundry portal:

1. Sign in to the [Azure AI Foundry portal](https://ai.azure.com) and open your project. 

1. In the **Overview** tile, find and copy the **Azure AI Foundry project endpoint**. 

   A hypothetical endpoint might look like this: `https://your-foundry-resource.services.ai.azure.com/api/projects/your-foundry-project`

## Load Connections

Load the environment variables to set up connections and object names.

In [43]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_endpoint = os.environ["PROJECT_ENDPOINT"]
project_connection_id = os.environ["PROJECT_CONNECTION_ID"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
agent_name = os.getenv("AGENT_NAME", "earth-knowledge-agent")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
search_api_key = os.environ["AZURE_SEARCH_API_KEY"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME", "earth-knowledge-source")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth-at-night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
base_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-knowledge-base")

## Create search index on Azure AI Search

This steps create a search index that contains plain text and vector content. You can use any existing search index, but it must meet the [criteria for agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schmea requirement is that is has a semantic configuration, with a `default_configuration_name`.

In [13]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")


Index 'earth-at-night' created or updated successfully


## Upload sample documents

This sample uses data from NASA's Earth at Night e-book. It's retrieved from the sample data GitHub repository and passed to the search client for indexing.

In [6]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")


Documents uploaded to index 'earth-at-night'


## Create a knowledge source

This step creates a knowledge source that targets the index you previously created. In the next step, you create a knowledge agent that uses the knowledge source to orchestrate agentic retrieval.


In [7]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters, SearchIndexFieldReference
from azure.search.documents.indexes import SearchIndexClient

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_fields=[SearchIndexFieldReference(name="id"), SearchIndexFieldReference(name="page_number")]
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'earth-knowledge-source' created or updated successfully.


## Create a knowledge base

This step creates a knowledge base, which acts as a wrapper for your knowledge source and LLM deployment.

`EXTRACTIVE_DATA` is the default modality and returns content from your knowledge sources without generative alteration. This is recommended for interaction with Foundry Agent Service

In [8]:
from azure.search.documents.indexes.models import KnowledgeBase, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeRetrievalOutputMode, KnowledgeRetrievalMinimalReasoningEffort
from azure.search.documents.indexes import SearchIndexClient

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_gpt_deployment,
    model_name=azure_openai_gpt_model,
)

knowledge_base = KnowledgeBase(
    name=base_name,
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name
        )
    ],
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
    retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort()
)


index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)
print(f"Knowledge base '{base_name}' created or updated successfully")

Knowledge base 'earth-knowledge-base' created or updated successfully


## Create an Azure AI Agent

In the Azure AI Foundry, an agent is a smart micro-service that can do RAG. The purpose of this specific agent is to decide when to send a query to the agentic retrieval pipeline.

In [19]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

list(project_client.agents.list())

[{'object': 'agent', 'id': 'earth-knowledge-agent', 'name': 'earth-knowledge-agent', 'versions': {'latest': {'metadata': {}, 'object': 'agent.version', 'id': 'earth-knowledge-agent:1', 'name': 'earth-knowledge-agent', 'version': '1', 'description': '', 'created_at': 1762328483, 'definition': {'kind': 'prompt', 'model': 'gpt-5-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nAlways provide references to the data source used to answer the question.\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [{'type': 'mcp', 'server_label': 'knowledge-base', 'server_url': 'https://magottei-s1m.search.windows.net/knowledgebases/earth-knowledge-base', 'require_approval': 'never'}]}}}},
 {'object': 'agent', 'id': 'fabric-foundry-knowledge-agent', 'name': 'fabric-foundry-knowledge-agent', 'versions': {'latest': {'metadata': {'logo': 'Avatar_Default.svg', 'description': '', 'modified_at': '1762295325', 'voiceLiveConfig': '{"isEnabled":fa

In [57]:
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

instructions = """
A Q&A agent that can answer questions about the Earth at night.
Always provide references to the data source used to answer the question.
If you do not have the answer, respond with "I don't know".
"""
mcp_kb_tool = MCPTool(
    server_label="knowledge-base",
    server_url=f"{endpoint}/knowledgebases/{base_name}/mcp?api-version=2025-11-01-Preview",
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id="kbmcpconnection",
    headers={
        "api-key": search_api_key
    }
)
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=agent_model,
        instructions=instructions,
        tools=[mcp_kb_tool]
    )
)


print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'earth-knowledge-agent' created or updated successfully


## Start a chat with the agent

In [58]:
# Get the OpenAI client for responses and conversations
openai_client = project_client.get_openai_client()

conversation = openai_client.conversations.create()

# Send initial request that will trigger the MCP tool
response = openai_client.responses.create(
    conversation=conversation.id,
    input="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Response: {response.output_text}")


InternalServerError: Error code: 500 - {'error': {'message': 'An error occurred while processing your request. You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. Please include the request ID 2706eba5-31ce-4cc0-9ce7-9697d4022216 in your message.', 'type': 'server_error', 'param': None, 'code': 'tool_server_error'}}

## Clean up objects and resources

If you no longer need the resources, be sure to delete them from your Azure subscription.  You can also delete individual objects to start over.

### Delete the agent

In [ ]:
index_client.delete_knowledge_base(agent_name)
print(f"Knowledge base '{agent_name}' deleted successfully")

Knowledge agent 'earth-search-agent' deleted successfully


### Delete the knowledge source

In [ ]:
index_client.delete_knowledge_source(knowledge_source=knowledge_source_name) # This is new feature in 2025-08-01-Preview api version
print(f"Knowledge source '{knowledge_source_name}' deleted successfully.")


Knowledge source 'earth-at-night-ks' deleted successfully.


### Delete the index

In [ ]:
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")

Index 'earth_at_night' deleted successfully
